In [1]:
import os
import torch

import utils

%load_ext autoreload
%autoreload 2

CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(os.listdir("../data/"))


['kawiki.txt', 'data3.txt', 'data.txt', 'data_115000.txt', 'data2.txt']


In [2]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
dataset = GeorgianLanguageDatasetLoader("../data/data_115000.txt", 5, device)

In [3]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

['<unk>', ',', '.', 'და', ')', '(', '„', '“', 'წელს', '—', 'იყო', 'წლის', ':', 'რომელიც', 'შემდეგ', 'რომ', 'მისი', '``', 'ამ', "''"]


In [4]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

tensor([ 910,  644,  517,  691, 9915,    1,   13,  154,  509, 1907])

In [5]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

'მინდა გემორჩილო და ბატონად გაღიაროვო “ . მსოფლიო ბანკი დანიას ევროკავშირში ბიზნესის ყველაზე ადვილად კეთების ადგილად მიიჩნევს . ქაოსი ('

In [6]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

[364, 342, 4857]

In [7]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [8]:
train_data_batched.shape, train_data_batched.device

(torch.Size([5, 158221]), device(type='cpu'))

In [9]:
x, y = utils.get_batch(train_data_batched, 1)

In [10]:
x.shape, y.shape, train_data_batched.shape

(torch.Size([5, 10]), torch.Size([5, 10]), torch.Size([5, 158221]))

In [11]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

True

In [12]:
x[0], y[0]

(tensor([ 644,  517,  691, 9915,    1,   13,  154,  509, 1907, 3464]),
 tensor([ 517,  691, 9915,    1,   13,  154,  509, 1907, 3464, 1095]))

In [13]:
%reload_ext autoreload

model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

/home/misho/anaconda3/envs/nlp_final_project/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


{'epoch': 0, 'batch': 0, 'loss': 11.65722942352295}
{'epoch': 1, 'batch': 0, 'loss': 11.572833061218262}
{'epoch': 2, 'batch': 0, 'loss': 11.498726844787598}
{'epoch': 3, 'batch': 0, 'loss': 11.41809368133545}
{'epoch': 4, 'batch': 0, 'loss': 11.350907325744629}
{'epoch': 5, 'batch': 0, 'loss': 11.2488431930542}
{'epoch': 6, 'batch': 0, 'loss': 11.152766227722168}
{'epoch': 7, 'batch': 0, 'loss': 11.028532981872559}
{'epoch': 8, 'batch': 0, 'loss': 10.830387115478516}
{'epoch': 9, 'batch': 0, 'loss': 10.58997917175293}
{'epoch': 10, 'batch': 0, 'loss': 10.226019859313965}
{'epoch': 11, 'batch': 0, 'loss': 9.704802513122559}
{'epoch': 12, 'batch': 0, 'loss': 9.00595474243164}
{'epoch': 13, 'batch': 0, 'loss': 8.119498252868652}
{'epoch': 14, 'batch': 0, 'loss': 7.096694469451904}
{'epoch': 15, 'batch': 0, 'loss': 6.022543430328369}
{'epoch': 16, 'batch': 0, 'loss': 5.041867256164551}
{'epoch': 17, 'batch': 0, 'loss': 4.189472198486328}
{'epoch': 18, 'batch': 0, 'loss': 3.599033832550049

In [20]:
text = utils.generate_text(model,device, dataset.vocab, 'იკვლევდა ევროპას და განსაკუთრებით', 20)

IndexError: list index out of range

In [15]:
print(text)

['იკვლევდა', 'ევროპას', 'და', 'განსაკუთრებით', 'ურიგებს', 'ასპარეზი', ',', 'კირქვისა', ',', 'ანდეზიტ-ბაზალტის', 'გურიელი', 'სამრეკლოს', ',', 'გურიელი', ',', 'და', 'სამრეკლოს', ',', 'გურიელი', 'სამრეკლოს', 'სამრეკლოს', 'კირქვისა', 'ანდეზიტ-ბაზალტის', 'ანდეზიტ-ბაზალტის']


In [16]:
torch.cuda.empty_cache()
# dataset = GeorgianLanguageDatasetLoader("../data/kawiki.txt", 5, device)

In [17]:
# geo_model = utils.GeorgianWord2VecModel(load=True)
# # geo_model.train("../data/kawiki.txt")

Initializing data
Model created!


In [18]:
# embeddings = torch.FloatTensor(geo_model.get_model().wv.vectors)
# # embeddings.shape

In [19]:
# %reload_ext autoreload
#
# model = utils.LSTMModel(128, 600, len(vocab), device, num_layers=1, embeddings=embeddings).cuda()
# model = model.to(device)
# utils.train_loop(model, train_data_batched, batch_size=5)

/home/misho/anaconda3/envs/nlp_final_project/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


{'epoch': 0, 'batch': 0, 'loss': 11.625279426574707}
{'epoch': 1, 'batch': 0, 'loss': 11.495043754577637}
{'epoch': 2, 'batch': 0, 'loss': 11.313353538513184}
{'epoch': 3, 'batch': 0, 'loss': 10.92991828918457}
{'epoch': 4, 'batch': 0, 'loss': 9.908686637878418}
{'epoch': 5, 'batch': 0, 'loss': 7.984778881072998}
{'epoch': 6, 'batch': 0, 'loss': 5.963374614715576}
{'epoch': 7, 'batch': 0, 'loss': 4.460447788238525}
{'epoch': 8, 'batch': 0, 'loss': 3.8317975997924805}
{'epoch': 9, 'batch': 0, 'loss': 3.6052162647247314}
{'epoch': 10, 'batch': 0, 'loss': 3.578481674194336}
{'epoch': 11, 'batch': 0, 'loss': 3.5857133865356445}
{'epoch': 12, 'batch': 0, 'loss': 3.607102632522583}
{'epoch': 13, 'batch': 0, 'loss': 3.566080331802368}
{'epoch': 14, 'batch': 0, 'loss': 3.5081725120544434}
{'epoch': 15, 'batch': 0, 'loss': 3.4336776733398438}
{'epoch': 16, 'batch': 0, 'loss': 3.3940911293029785}
{'epoch': 17, 'batch': 0, 'loss': 3.3870580196380615}
{'epoch': 18, 'batch': 0, 'loss': 3.3852837085

KeyboardInterrupt: 

In [ ]:
# import torchtext
# v = torchtext.vocab.vocab(geo_model.get_model().wv.key_to_index, specials=['<unk>'])
# v.set_default_index(v['<unk>'])
# # dataset.vocab.load_state_dict(geo_model.get_model().wv.key_to_index)
# text = utils.generate_text(model,device, v, 'იკვლევდა ევროპას და განსაკუთრებით', 10)
